In [89]:
import random
import time
import math
from queue import *
import numpy as np
import matplotlib.pyplot as plt

In [1]:
# board = np.zeros((3,3), dtype=int)

def printBoard():
    print("   0    1    2")
    print("  ------------")
    for i in range(len(board)):
        print(i, end=" | ")
        print(board[i,0], end="")
        print(" | ", end="")
        print(board[i,1], end="")
        print(" | ", end="")
        print(board[i,2], end=" |\n")
        print("  ------------")
        
        
# printBoard()
import pprint
perms = [
    [
        0, 1, 2,
        3, 4, 5,
        6, 7, 8
    ],
    [
        2, 5, 8,
        1, 4, 7,
        0, 3, 6
    ],
    [
        8, 7, 6,
        5, 4, 3,
        2, 1, 0
    ],
    [
        6, 3, 0,
        7, 4, 1,
        8, 5, 2
    ],
    [
        2, 1, 0,
        5, 4, 3,
        8, 7, 6
    ],
    [
        0, 3, 6,
        1, 4, 7,
        2, 5, 8
    ],
    [
        6, 7, 8,
        3, 4, 5,
        0, 1, 2
    ],
    [
        8, 5, 2,
        7, 4, 1,
        6, 3, 0
    ]
]
powers = [[3 ** i for i in perm] for perm in perms]
pprint.pprint(powers)


[[1, 3, 9, 27, 81, 243, 729, 2187, 6561],
 [9, 243, 6561, 3, 81, 2187, 1, 27, 729],
 [6561, 2187, 729, 243, 81, 27, 9, 3, 1],
 [729, 27, 1, 2187, 81, 3, 6561, 243, 9],
 [9, 3, 1, 243, 81, 27, 6561, 2187, 729],
 [1, 27, 729, 3, 81, 2187, 9, 243, 6561],
 [729, 2187, 6561, 27, 81, 243, 1, 3, 9],
 [6561, 243, 9, 2187, 81, 3, 729, 27, 1]]


In [91]:
def checkWins(board):
    for line in board:
        if all(item == 1 for item in line):
            return 1
        if all(item == 2 for item in line):
            return 2
    for i in range(len(board)):
        if all(item == 1 for item in board[:,i]):
            return 1
        if all(item == 2 for item in board[:,i]):
            return 2
    if all(item == 1 for item in board.diagonal()):
        return 1
    if all(item == 2 for item in board.diagonal()):
        return 2
    if board[2,0] == 1 and board[1,1] == 1 and board[0,2] == 1:
        return 1
    if board[2,0] == 2 and board[1,1] == 2 and board[0,2] == 2:
        return 2
    return 0



def gameTurn(player):
    while True:
        print("Player", player)
        x = int(input("Choose y position: (0,1,2) "))
        y = int(input("Choose x position: (0,1,2) "))
        if x > 2 or y > 2:
            print("Out of bounds")
            continue
        if board[x,y] == 0:
            board[x,y] = player
            return checkWins(board)
        print("There is already a chip in this place")
    
def game():
    board = np.zeros((3,3), dtype=int)
    winner = 0
    while True:
        for i in range(1,3):
            
            
            
            winner = gameTurn(i)
            printBoard()
            if winner > 0:
                print("Player", winner, "wins!")
                return winner
            
            


In [93]:
def reward(playerNumber, board):
    winner = 0
    for line in board:
        if all(item == 1 for item in line):
            winner = 1
        if all(item == 2 for item in line):
            winner = 2
    for i in range(len(board)):
        if all(item == 1 for item in board[:,i]):
            winner = 1
        if all(item == 2 for item in board[:,i]):
            winner = 2
    if all(item == 1 for item in board.diagonal()):
        winner = 1
    if all(item == 2 for item in board.diagonal()):
        winner = 2
    if board[2,0] == 1 and board[1,1] == 1 and board[0,2] == 1:
        winner = 1
    if board[2,0] == 2 and board[1,1] == 2 and board[0,2] == 2:
        winner = 2
    if winner == playerNumber:
        return 1
    elif winner != 0:
        return -1
    return 0

def checkPossibleActions(state):
    actions = []
    for i in range(state):
        for j in range(state[i]):
            if state[i,j] == 0:
                actions.append([i,j])
    return actions
                

def placeChip(state, chipLocation, playerNumber):
    state[chipLocation[0], chipLocation[1]] = playerNumber
    return state
    
def policy(state, utilities, epsilon, playerNumber):
    actions = checkPossibleActions(state)
    actionReward = -2
    currentAction = None
    if random.random() < epsilon:
        return random.choice(actions)
    for action in actions:
        tempState = placeChip(state, action, playerNumber)
        tempStatekey = hashState(tempState)
        if tempStateKey in utilities:
            if utilities[tempStatekey] >= actionReward:
                currentAction = action
                actionReward = utilities[tempStatekey]
    if currentAction == None: return random.choice(actions)
    return currentAction


def hashState(board):
    return '\n'.join('\t'.join('%0.3f' %x for x in y) for y in board)

def learningAgent(playerNumber, board):
    utilities = {}
    ns = {}
#     b = '\n'.join('\t'.join('%0.3f' %x for x in y) for y in board)
    alpha = 0.0001
    gamma = 1
    plays = 0
    totalPlays = 0
    gameCount = 0
    state = board
    lastState = None
    stateQueue = Queue()
    while True:
        plays += 1
        epsilon = max(1 - (0.00001 * plays), 0.05)
        key = hashState(state)
        if key in ns == False:
            ns[key] = 0
            utilities[key] = reward(state)
        if lastState:
            ns[lastBoardKey] += 1
            alpha = 60/(59 + ns[lastBoardKey])
            queueCount = 1
            for i in range(stateQueue.qsize()):
                queuedState = stateQueue.get()
                qStateKey = hashState(queuedState)
                utilities[qStateKey] +=\
                                    (\
                                      alpha * \
                                      (\
                                          rewardGrid[qStateKey]\
                                          + gamma\
                                          * utilities[key]\
                                          - utilities[qStateKey]\
                                      )\
                                    )\
                                    / queueCount
                queueCount += queueCount
                stateQueue.put(queuedState)
        stateQueue.put(state)
        if reward(state) != 0:
            gameCount += 1
            totalPlays += plays
            plays = 0
            stateQueue.queue.clear()
            if gameCount != 1000:
                state = board
                continue
            return totalPlays
        else:
            lastState = state
            action = policy(state, utilities, epsilon)
            state = placeChip(state, action, playerNumber)
            #action[1] is where the chip was placed
            
            
            
                
                
    pass


Choose y position: (0,1,2) 0
Choose x position: (0,1,2) 0
   0    1    2
  ------------
0 | 1 | 0 | 0 |
  ------------
1 | 0 | 0 | 0 |
  ------------
2 | 0 | 0 | 0 |
  ------------
Choose y position: (0,1,2) 3
Choose x position: (0,1,2) 4
Out of bounds
Choose y position: (0,1,2) 1
Choose x position: (0,1,2) 1
   0    1    2
  ------------
0 | 1 | 0 | 0 |
  ------------
1 | 0 | 2 | 0 |
  ------------
2 | 0 | 0 | 0 |
  ------------
Choose y position: (0,1,2) 0
Choose x position: (0,1,2) 0
There is already a chip in this place
Choose y position: (0,1,2) 0
Choose x position: (0,1,2) 1
   0    1    2
  ------------
0 | 1 | 1 | 0 |
  ------------
1 | 0 | 2 | 0 |
  ------------
2 | 0 | 0 | 0 |
  ------------
Choose y position: (0,1,2) 0
Choose x position: (0,1,2) 0
There is already a chip in this place
Choose y position: (0,1,2) 2
Choose x position: (0,1,2) 0
   0    1    2
  ------------
0 | 1 | 1 | 0 |
  ------------
1 | 0 | 2 | 0 |
  ------------
2 | 2 | 0 | 0 |
  ------------
Choose y pos